In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
from sklearn.metrics import make_scorer, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import preprocessing
import xgboost as xgb
import scipy.stats as stats
import seaborn as sns
import gc

from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

In [ ]:
train = pd.read_csv('grupo-bimbo-inventory-demand/train.csv')
test = pd.read_csv('grupo-bimbo-inventory-demand/test.csv')

print('Train and Test Read')

In [ ]:

train['target'] = train['Demanda_uni_equil']
train.drop(['Demanda_uni_equil'],axis=1, inplace = True)

train['tst'] = 0
test['tst'] = 1

data = pd.concat([train,test], axis=0, copy=True)

print('Train and Test Concat')
del train
del test
gc.collect()

In [ ]:
for i in range(1,6):
    lag = 'Lag' + str(i)
    print('Lag:',lag)

    data1 = data[['Semana','Cliente_ID','Producto_ID','target']]
    data1.loc[:,'Semana'] = data1['Semana'] +i
    data1 = data1.groupby(['Semana','Cliente_ID','Producto_ID']).mean()
    data1 = data1.reset_index()
    data1.rename(columns={'target': lag}, inplace=True)
    data = pd.merge(data,data1,
                    how='left',
                    left_on=['Semana','Cliente_ID','Producto_ID'],
                    right_on=['Semana','Cliente_ID','Producto_ID'],
                    left_index=False, right_index=False, sort=True,
                    suffixes=('_x', '_y'), copy=False, )
    del data1
    gc.collect()

In [ ]:

data['TotalLags'] = data['Lag1'] + data['Lag2']+ data['Lag3']+ data['Lag4']+ data['Lag5']

data=data[data['Semana']>8]  # NOW I WORK WITH WEEKS 9,10,11

nAgencia = pd.DataFrame(data.groupby(['Agencia_ID','Semana'])['target'].count())
nAgencia = nAgencia.reset_index()
nAgencia.rename(columns={'target': 'nAgencia'}, inplace=True)
nAgencia = pd.DataFrame(nAgencia.groupby(['Agencia_ID'])['nAgencia'].mean())
nAgencia = nAgencia.reset_index()


data = pd.merge(data, nAgencia,
                how='left',
                left_on=['Agencia_ID'],
                right_on=['Agencia_ID'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nAgencia
gc.collect()
print('merge completo nAgencia')
print(data.shape[0])

nRuta_SAK = pd.DataFrame(data.groupby(['Ruta_SAK','Semana'])['target'].count())
nRuta_SAK = nRuta_SAK.reset_index()
nRuta_SAK.rename(columns={'target': 'nRuta_SAK'}, inplace=True)
nRuta_SAK = pd.DataFrame(nRuta_SAK.groupby(['Ruta_SAK'])['nRuta_SAK'].mean())
nRuta_SAK = nRuta_SAK.reset_index()


data = pd.merge(data, nRuta_SAK,
                how='left',
                left_on=['Ruta_SAK'],
                right_on=['Ruta_SAK'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nRuta_SAK
gc.collect()
print('merge completo nRuta_SAK')
print(data.shape[0])

nCliente_ID = pd.DataFrame(data.groupby(['Cliente_ID','Semana'])['target'].count())
nCliente_ID = nCliente_ID.reset_index()
nCliente_ID.rename(columns={'target': 'nCliente_ID'}, inplace=True)
nCliente_ID = pd.DataFrame(nCliente_ID.groupby(['Cliente_ID'])['nCliente_ID'].mean())
nCliente_ID = nCliente_ID.reset_index()


data = pd.merge(data, nCliente_ID,
                how='left',
                left_on=['Cliente_ID'],
                right_on=['Cliente_ID'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nCliente_ID
gc.collect()
print('merge completo nCliente_ID')
print(data.shape[0])

nProducto_ID = pd.DataFrame(data.groupby(['Producto_ID','Semana'])['target'].count())
nProducto_ID = nProducto_ID.reset_index()
nProducto_ID.rename(columns={'target': 'nProducto_ID'}, inplace=True)
nProducto_ID = pd.DataFrame(nProducto_ID.groupby(['Producto_ID'])['nProducto_ID'].mean())
nProducto_ID = nProducto_ID.reset_index()


data = pd.merge(data, nProducto_ID,
                how='left',
                left_on=['Producto_ID'],
                right_on=['Producto_ID'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nProducto_ID
gc.collect()
print('merge completo nProducto_ID')
print(data.shape[0])

data.replace(np.nan,0, inplace=True)

train = data[data['tst']==0]
predict = data[data['tst']==1]

train['target'] = np.log(train['target'] + 1)
#train2 = train.sample(n=1000000)   <-- another possible reduction of data for fast testing
train2 = train
y = train['target']
X = train[[  'Agencia_ID','Canal_ID','Cliente_ID','Producto_ID','Ruta_SAK',
             'Lag1','Lag2','Lag3','Lag4','Lag5','TotalLags',
             'nAgencia','nRuta_SAK','nCliente_ID','nProducto_ID']]

print(X.shape, y.shape)

In [ ]:
# X.to_csv('x.csv',index=False)
# y.to_csv('y.csv',index=False)
train.to_csv('data.csv',index=False)

In [ ]:
# sns.palplot(train)

# Training

In [19]:
X=pd.read_csv('x.csv')
y=pd.read_csv('y.csv')
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=1729)
print(X_train.shape, X_test.shape)

(10304625, 15) (104088, 15)


In [37]:
xlf = xgb.XGBRegressor(max_depth=10,
                       learning_rate=0.1,
                       objective="reg:squarederror",
                       nthread=-1,
                       gamma=0,
                       seed=1440,
                       missing=1,
                       eval_metric='rmsle')

In [4]:
xlf.fit(X_train, y_train, verbose = True, eval_set = [(X_test, y_test)])

[15:32:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmsle:0.51802
[1]	validation_0-rmsle:0.45946
[2]	validation_0-rmsle:0.41249
[3]	validation_0-rmsle:0.37411
[4]	validation_0-rmsle:0.34310
[5]	validation_0-rmsle:0.31783
[6]	validation_0-rmsle:0.29627
[7]	validation_0-rmsle:0.27899
[8]	validation_0-rmsle:0.26432
[9]	validation_0-rmsle:0.25294


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmsle', gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
             missing=1, monotone_constraints='()', n_estimators=10, n_jobs=-1,
             nthread=-1, num_parallel_tree=1, predictor='auto',
             random_state=1440, reg_alpha=0, ...)

In [12]:
# calculate the auc score
preds = xlf.predict(X_test)

print('Validation RMSLE XDBoost ', mean_squared_log_error(y_test,preds, squared=False))

Validation RMSLE XDBoost  0.25294002423995865


In [10]:
lgbm_regr = lgb.LGBMRegressor(num_leaves=1000,
                              max_depth=10,
                              random_state=42)

lgbm_regr.fit(X_train,y_train,verbose=True,eval_set=(X_test,y_test),eval_metric='rmsle')

C:\Anaconda\envs\torch\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.627009
[2]	valid_0's l2: 0.557831
[3]	valid_0's l2: 0.501662
[4]	valid_0's l2: 0.455631
[5]	valid_0's l2: 0.418407
[6]	valid_0's l2: 0.38801
[7]	valid_0's l2: 0.362009
[8]	valid_0's l2: 0.340101
[9]	valid_0's l2: 0.32279
[10]	valid_0's l2: 0.307942
[11]	valid_0's l2: 0.296124
[12]	valid_0's l2: 0.286203
[13]	valid_0's l2: 0.277763
[14]	valid_0's l2: 0.270854
[15]	valid_0's l2: 0.264717
[16]	valid_0's l2: 0.259782
[17]	valid_0's l2: 0.255866
[18]	valid_0's l2: 0.251835
[19]	valid_0's l2: 0.248745
[20]	valid_0's l2: 0.245626
[21]	valid_0's l2: 0.243485
[22]	valid_0's l2: 0.241481
[23]	valid_0's l2: 0.23939
[24]	valid_0's l2: 0.237606
[25]	valid_0's l2: 0.236359
[26]	valid_0's l2: 0.235169
[27]	valid_0's l2: 0.233922
[28]	valid_0's l2: 0.232997
[29]	valid_0's l2: 0.232219
[30]	valid_0's l2: 0.231336
[31]	valid_0's l2: 0.230564
[32]	valid_0's l2: 0.229731
[33]	valid_0's l2: 0.22895
[34]	valid_0's l2: 0.228189
[35]	valid_0's l2: 0.227583
[36]	valid_0's l2: 0.226809
[37]	

LGBMRegressor(max_depth=10, num_leaves=1000, random_state=42)

In [36]:
preds = lgbm_regr.predict(X_test)

drop_indeces=np.where(preds<0)
preds=np.delete(preds,drop_indeces)
y_test_l=np.delete(y_test.values,drop_indeces)

print('Validation RMSLE LGBM Decision tree', mean_squared_log_error(y_test_l,preds, squared=False))

Validation RMSLE LGBM Decision tree 0.19593781047373218
